In [11]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os
from pathlib import Path

import mlflow
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from matplotlib import pyplot as plt
import xgboost as xgb


In [12]:
DATA_PAth = '../data/processed/data_engineered_df.pkl'
TRACKING_URI = "../mlflow"
LOG_DATA_PKL    =  "data.pkl"
EXPERIMENT_NAME = "job-profile-prediction"



In [13]:
# quality metrics for model evaluation
def calculate_quality(actual, predictions):
    metrics = [accuracy_score, precision_score, recall_score, f1_score]
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = actual[col].copy()
        quality_scores[col] = [round(metric_function(role_truth, role_pred) * 100, 2) for metric_function in metrics]
    
    indecies = [score.__name__ for score in metrics]
    quality_scores = pd.DataFrame(quality_scores, index=indecies).T
    
    return quality_scores

In [14]:
df = pd.read_pickle(DATA_PAth)

In [15]:
skills = df.drop(['DevType'], axis=1).copy()
jobs =  df['DevType'].copy()



/var/folders/18/bfxv49yn02qg3c3v6g5wgy7w0000gn/T/ipykernel_91164/3352739922.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  skills = df.drop(['DevType'], axis=1).copy()


## Resampling

In [16]:
# Resample roles
samples_per_class = 1500
resampled_roles = []

for role_col in jobs.columns:
    sub_df = jobs.loc[jobs[role_col] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class+1000, replace=True, random_state=0) 
    
    resampled_roles.append(sub_df)

In [17]:
roles_df  = pd.concat(resampled_roles)
df = df.loc[roles_df.index].copy()

## train_test_split

In [18]:
skills = df.drop(['DevType'], axis=1).copy()
jobs =  df['DevType'].copy() 
X_train, X_test, Y_train, Y_test = train_test_split(skills, 
                                                    jobs, 
                                                    random_state=0)

/var/folders/18/bfxv49yn02qg3c3v6g5wgy7w0000gn/T/ipykernel_91164/470520421.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  skills = df.drop(['DevType'], axis=1).copy()


## train xgboost model

In [19]:
xgb_model = make_pipeline(
                      RobustScaler(),
                       PCA(n_components=0.7),
                       xgb.XGBClassifier(),
                                            )

fit = xgb_model.fit(X_train, Y_train)


## evaluate model

In [20]:
y_pred = pd.DataFrame(xgb_model.predict(X_train),
                            columns=Y_train.columns)
xgb_train_scores = calculate_quality(Y_train, y_pred)
                
print(xgb_train_scores.mean())
xgb_train_scores


accuracy_score     99.973830
precision_score    99.792766
recall_score       99.276809
f1_score           99.526383
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
Blockchain,99.99,99.65,100.00,99.83
Cloud infrastructure engineer,100.00,99.91,99.82,99.87
Data or business analyst,99.97,99.46,99.01,99.23
Data scientist or machine learning specialist,99.99,100.00,99.57,99.79
Database administrator,99.98,98.95,99.73,99.34
DevOps specialist,100.00,100.00,100.00,100.00
"Developer, QA or test",99.99,99.73,99.56,99.64
"Developer, back-end",100.00,99.91,100.00,99.96
"Developer, desktop or enterprise applications",99.63,99.30,88.54,93.61
"Developer, embedded applications or devices",99.91,99.42,98.93,99.17


In [21]:
y_pred = pd.DataFrame(xgb_model.predict(X_test),
                            columns=Y_test.columns)
xgb_test_scores = calculate_quality(Y_test, y_pred)
                
print(xgb_test_scores.mean())
xgb_test_scores


accuracy_score     99.531277
precision_score    95.940000
recall_score       84.623617
f1_score           89.280213
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
Blockchain,99.99,99.71,99.71,99.71
Cloud infrastructure engineer,99.52,98.63,75.39,85.46
Data or business analyst,99.57,94.97,81.89,87.95
Data scientist or machine learning specialist,99.52,93.92,74.62,83.16
Database administrator,99.96,99.19,98.65,98.92
DevOps specialist,99.37,96.79,66.39,78.76
"Developer, QA or test",99.87,99.43,93.57,96.41
"Developer, back-end",100.00,99.74,100.00,99.87
"Developer, desktop or enterprise applications",98.23,91.19,40.96,56.53
"Developer, embedded applications or devices",99.35,97.25,91.67,94.38


## random forest model

In [22]:
random_forest = make_pipeline(
                       PCA(n_components=0.7),
                       RandomForestClassifier(n_jobs=8,
                                              verbose=1,
                                              random_state=0,
                                              n_estimators=500,
                                              ))

random_forest.fit(X_train.values, Y_train.values)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   10.4s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   51.6s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:  2.4min finished


Pipeline(steps=[('pca', PCA(n_components=0.7)),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=500, n_jobs=8,
                                        random_state=0, verbose=1))])

## evaluate model

In [23]:
y_pred = pd.DataFrame(random_forest.predict(X_train),
                            columns=Y_train.columns)
rf_train_scores = calculate_quality(Y_train, y_pred)
                
print(rf_train_scores.mean())
rf_train_scores

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   14.6s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   33.5s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   38.0s finished


accuracy_score     99.991702
precision_score    99.828298
recall_score       99.801915
f1_score           99.815532
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
Blockchain,99.99,99.65,100.00,99.83
Cloud infrastructure engineer,100.00,99.91,99.82,99.87
Data or business analyst,99.97,99.64,98.92,99.28
Data scientist or machine learning specialist,99.99,100.00,99.57,99.79
Database administrator,99.98,98.95,99.73,99.34
DevOps specialist,100.00,100.00,100.00,100.00
"Developer, QA or test",99.99,99.73,99.65,99.69
"Developer, back-end",100.00,99.91,100.00,99.96
"Developer, desktop or enterprise applications",99.96,99.79,98.91,99.35
"Developer, embedded applications or devices",99.96,99.51,99.83,99.67


In [24]:
y_pred = pd.DataFrame(random_forest.predict(X_test),
                            columns=Y_test.columns)
rf_test_scores = calculate_quality(Y_test, y_pred)
                
print(rf_test_scores.mean())
rf_test_scores

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    3.9s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    9.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   10.4s finished


accuracy_score     99.531489
precision_score    98.992340
recall_score       82.051702
f1_score           88.539149
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
Blockchain,99.99,99.71,99.71,99.71
Cloud infrastructure engineer,99.53,100.00,74.87,85.63
Data or business analyst,99.62,98.76,81.12,89.08
Data scientist or machine learning specialist,99.41,98.17,64.65,77.96
Database administrator,99.96,99.19,98.65,98.92
DevOps specialist,99.40,100.00,65.84,79.40
"Developer, QA or test",99.87,99.43,93.57,96.41
"Developer, back-end",100.00,100.00,100.00,100.00
"Developer, desktop or enterprise applications",98.40,97.36,44.41,60.99
"Developer, embedded applications or devices",99.44,98.11,92.48,95.21


In [27]:
# Define the parameter grid
param_grid = {
    'xgbclassifier__n_estimators': [100, 200, 300],
    'xgbclassifier__max_depth': [3, 4, 5],
    'xgbclassifier__learning_rate': [0.1, 0.01, 0.001],
     'pca__n_components': [0.7, 0.85, 0.95]

}

hpt_pipeline = make_pipeline(RobustScaler(),
                           PCA(),
                            xgb.XGBClassifier()
                        )

# Perform grid search
grid_search = GridSearchCV(hpt_pipeline, param_grid=param_grid, cv=5)
grid_search.fit(X_train, Y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)


KeyboardInterrupt: 

In [28]:
data_details = {
    "features_names": X_train.columns.droplevel(0).tolist(),
    "targets_names": Y_train.columns.tolist()
}

In [29]:
os.environ["MLFLOW_TRACKING_URI"] = TRACKING_URI

exp_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id


In [30]:


# Start a new run and track 
with mlflow.start_run(experiment_id=exp_id, run_name="xgboost"):
    # Log pickles 
    mlflow.sklearn.log_model(xgb_model, "model", registered_model_name="xgboost") 
    [mlflow.log_metric(score, value) for score, value in rf_test_scores.mean().items()]
    mlflow.log_dict(data_details, "data/data_details")

    

/Users/Aya/anaconda3/envs/de/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'xgboost'.
Created version '1' of model 'xgboost'.
